In [1]:
import sys
import os
from pathlib import Path
current_dir = os.getcwd()  
project_root = os.path.dirname(current_dir)  
sys.path.insert(0, project_root) 
import numpy as np
import pandas as pd
from src.data.data_preprocessor import DataPreprocessor
from src.config.config import Config
data_preprocessor = DataPreprocessor()

def detect_and_remove_anomalies(df, target_col, country_col='Country Name', year_col='Year', 
                               z_threshold=2.0, ratio_threshold=2.0, 
                               min_years=3, return_anomalies=False):
    """
    检测并移除目标列中的异常值 - 改进版
    
    参数:
    - df: 数据框
    - target_col: 目标列名（工业废物量）
    - country_col: 国家列名
    - year_col: 年份列名
    - z_threshold: Z-score阈值，超过此值视为异常
    - ratio_threshold: 比值阈值，与相邻年份比值超过此值视为异常
    - min_years: 最少需要的年份数据量，少于此值的国家不进行异常检测
    - return_anomalies: 是否返回异常值信息
    
    返回:
    - 清洗后的数据框，如果return_anomalies=True，则同时返回异常值信息
    """
    # 复制数据框，避免修改原始数据
    df_clean = df.copy()
    
    # 存储所有异常记录
    all_anomalies = pd.DataFrame()
    
    # 按国家分组处理
    for country, country_data in df_clean.groupby(country_col):
        # 如果数据量太少，跳过异常检测
        if len(country_data) < min_years:
            continue
            
        # 按年份排序
        country_data = country_data.sort_values(year_col)
        
        # 方法1: 基于Z-score的异常检测
        z_scores = np.abs((country_data[target_col] - country_data[target_col].mean()) / country_data[target_col].std())
        z_anomalies = country_data[z_scores > z_threshold].index
        
        # 方法2: 基于相邻年份比值的异常检测
        country_data_sorted = country_data.sort_values(year_col)
        
        # 计算与前一年的比值
        prev_ratios = country_data_sorted[target_col] / country_data_sorted[target_col].shift(1)
        # 计算与后一年的比值
        next_ratios = country_data_sorted[target_col] / country_data_sorted[target_col].shift(-1)
        
        # 找出异常记录
        ratio_anomalies = country_data_sorted[
            ((prev_ratios > ratio_threshold) | (prev_ratios < 1/ratio_threshold)) & 
            ((next_ratios > ratio_threshold) | (next_ratios < 1/ratio_threshold))
        ].index
        
        # 方法3: 考虑年份间隔的异常检测
        # 计算年份间隔
        year_gaps = country_data_sorted[year_col].diff()
        # 调整比值阈值，根据年份间隔
        adjusted_ratios = prev_ratios / year_gaps
        gap_anomalies = country_data_sorted[
            (adjusted_ratios > ratio_threshold) & (year_gaps > 1)
        ].index
        
        # 方法4: 基于IQR的异常检测 (新增)
        Q1 = country_data[target_col].quantile(0.25)
        Q3 = country_data[target_col].quantile(0.75)
        IQR = Q3 - Q1
        iqr_lower_bound = Q1 - 1.5 * IQR
        iqr_upper_bound = Q3 + 1.5 * IQR
        iqr_anomalies = country_data[
            (country_data[target_col] < iqr_lower_bound) | 
            (country_data[target_col] > iqr_upper_bound)
        ].index
        
        # 方法5: 基于移动平均的异常检测 (新增)
        if len(country_data) >= 5:  # 至少需要5个数据点
            # 计算移动平均和标准差
            rolling_mean = country_data[target_col].rolling(window=3, min_periods=1).mean()
            rolling_std = country_data[target_col].rolling(window=3, min_periods=1).std().fillna(country_data[target_col].std())
            
            # 检测时间序列异常值
            lower_bound = rolling_mean - 2 * rolling_std
            upper_bound = rolling_mean + 2 * rolling_std
            
            # 创建异常值掩码
            rolling_anomalies = country_data[
                (country_data[target_col] < lower_bound) | 
                (country_data[target_col] > upper_bound)
            ].index
        else:
            rolling_anomalies = pd.Index([])
        
        # 方法6: 基于增长率的异常检测 (新增)
        # 计算年增长率
        growth_rates = country_data[target_col].pct_change() * 100
        # 异常增长率阈值 (例如超过50%或下降超过30%)
        growth_anomalies = country_data[
            (growth_rates > 50) | (growth_rates < -30)
        ].index
        
        # 综合多种方法，确定最终的异常值
        # 使用投票机制：至少被两种方法判定为异常的记录
        anomaly_counts = {}
        for idx in set(z_anomalies) | set(ratio_anomalies) | set(gap_anomalies) | set(iqr_anomalies) | set(rolling_anomalies) | set(growth_anomalies):
            anomaly_counts[idx] = 0
            if idx in z_anomalies: anomaly_counts[idx] += 1
            if idx in ratio_anomalies: anomaly_counts[idx] += 1
            if idx in gap_anomalies: anomaly_counts[idx] += 1
            if idx in iqr_anomalies: anomaly_counts[idx] += 1
            if idx in rolling_anomalies: anomaly_counts[idx] += 1
            if idx in growth_anomalies: anomaly_counts[idx] += 1
        
        # 至少被两种方法判定为异常
        anomaly_indices = [idx for idx, count in anomaly_counts.items() if count >= 2]
        
        if anomaly_indices and return_anomalies:
            # 收集异常记录信息
            anomalies = country_data.loc[anomaly_indices].copy()
            anomalies['z_score'] = z_scores.loc[anomaly_indices]
            anomalies['prev_ratio'] = prev_ratios.loc[anomaly_indices]
            anomalies['next_ratio'] = next_ratios.loc[anomaly_indices]
            anomalies['year_gap'] = year_gaps.loc[anomaly_indices]
            # 添加新的异常检测指标
            anomalies['iqr_lower'] = iqr_lower_bound
            anomalies['iqr_upper'] = iqr_upper_bound
            if len(country_data) >= 5:
                anomalies['rolling_lower'] = lower_bound.loc[anomaly_indices]
                anomalies['rolling_upper'] = upper_bound.loc[anomaly_indices]
            anomalies['growth_rate'] = growth_rates.loc[anomaly_indices]
            anomalies['anomaly_methods'] = [anomaly_counts[idx] for idx in anomaly_indices]
            all_anomalies = pd.concat([all_anomalies, anomalies])
        
        # 从清洗数据中移除异常值
        df_clean = df_clean.drop(anomaly_indices)
    
    if return_anomalies:
        return df_clean, all_anomalies
    else:
        return df_clean

In [2]:
"""加载1990-2022数据"""
historical_df = pd.read_excel(
    Config.FEATURE_CONFIG['historical_data_path'],
    sheet_name=Config.FEATURE_CONFIG['historical_sheet'],
    usecols=Config.FEATURE_CONFIG['usecols'] 
)

historical_df

,Year,Country Name,Population,GDP PPP 2017,GDP PPP/capita 2017,Urban population %,Region,Income Group
0,1990,Spain,38867322,1.070531e+12,27543.22,75.351,Europe & Central Asia,High income
1,1991,Spain,38966376,1.097787e+12,28172.67,75.528,Europe & Central Asia,High income
2,1992,Spain,39157685,1.107988e+12,28295.54,75.610,Europe & Central Asia,High income
3,1993,Spain,39361262,1.096559e+12,27858.84,75.692,Europe & Central Asia,High income
4,1994,Spain,39549108,1.122692e+12,28387.29,75.774,Europe & Central Asia,High income
...,...,...,...,...,...,...,...,...
5426,2018,Switzerland,8514329,5.928471e+11,69629.34,73.797,Europe & Central Asia,High income
5427,2019,Switzerland,8575280,5.996173e+11,69923.93,73.849,Europe & Central Asia,High income
5428,2020,Switzerland,8638167,5.867751e+11,67928.19,73.915,Europe & Central Asia,High income
5429,2021,Switzerland,8704546,6.184133e+11,71044.87,73.996,Europe & Central Asia,High income


In [3]:
"""执行完整处理流程"""
# 第一阶段：处理历史数据
all_countries_df = data_preprocessor.process_historical_data(historical_df)
features_path = Path(Config.PATH_CONFIG['features_dir']) / 'global_features.csv'
all_countries_df.to_csv(features_path, index=False)

正在计算全局统计量...
...计算完成: GDP PPP 2017
...计算完成: GDP PPP/capita 2017
...计算完成: Population
...计算完成: Urban population %
Fit 过程完成。全局统计量已计算（如果找到列）。
开始 Transform 过程...
应用对数变换...
...已生成: GDP PPP 2017_log
...已生成: GDP PPP/capita 2017_log
...已生成: Population_log
创建非线性特征...
...已生成: gdp_pc_log_squared
创建时间特征...
...已生成: year_relative (基准年: 1990)
计算增长率...
...已生成: GDP PPP 2017_log_growth_1y
...已生成: Population_log_growth_1y
处理城市化率...
...已生成: urban_pop_perc
创建交互特征...
...已生成: gdp_log_x_pop_log
...已生成: gdp_pc_log_x_urban
...已生成: gdp_pc_log2_x_urban
创建时间相关的交互特征...
...已生成: year_relative_x_gdp_pc_log
...已生成: year_relative_x_Population_log
...已生成: year_relative_x_urban_pop_perc
处理缺失值和无穷值...
...正在使用中位数填充NaN值...
...填充完成，共处理了 0 个 NaN 值。
裁剪极端数值...
Transform 过程完成。
特征工程参数已保存至: e:\code\jupyter\固废产生\SW-Prediction\iw\src\features\featurefile\feature_params.pkl


In [4]:
# 第二阶段：处理MSW数据
"""加载包含MSW的目标数据"""
msw_df = pd.read_excel(
    Config.FEATURE_CONFIG['historical_msw_data_path'],
    sheet_name=Config.FEATURE_CONFIG['historical_msw_sheet'],
    usecols=Config.FEATURE_CONFIG['usecols'] + [Config.DATA_CONFIG['target_column']]
)

# 先检测异常值但不移除，查看异常值情况
target_col = Config.DATA_CONFIG['target_column']
_, anomalies = detect_and_remove_anomalies(msw_df, target_col, return_anomalies=True)
print(f"检测到 {len(anomalies)} 条异常记录:")
print(f"检测到 {len(anomalies)} 条异常记录:")
if len(anomalies) > 0:
    display(anomalies.sort_values(['Country Name', 'Year']))
else:
    print("没有检测到异常记录")

# 分析每个国家数据的分布情况
print("\n===== 各国数据分布情况分析 =====")
for country, country_data in msw_df.groupby('Country Name'):
    if len(country_data) < 3:  # 跳过数据量太少的国家
        continue
        
    country_data = country_data.sort_values('Year')
    
    # 计算Z-score
    z_scores = np.abs((country_data[target_col] - country_data[target_col].mean()) / country_data[target_col].std())
    
    # 计算年度增长比例
    growth_ratios = country_data[target_col].pct_change()
    
    print(f"\n国家: {country}, 数据点数: {len(country_data)}")
    print(f"平均值: {country_data[target_col].mean():.2f}, 标准差: {country_data[target_col].std():.2f}")
    print(f"最大Z-score: {z_scores.max():.2f}")
    print(f"最大年度增长率: {growth_ratios.max():.2%}")
    print(f"最小年度增长率: {growth_ratios.min():.2%}")
    
    # 显示可能的异常点
    potential_anomalies = country_data[(z_scores > 2.0) | (growth_ratios > 0.5) | (growth_ratios < -0.3)]
    if len(potential_anomalies) > 0:
        print("潜在异常点:")
        display(potential_anomalies[['Year', target_col]])


检测到 9 条异常记录:
检测到 9 条异常记录:


,Year,Country Name,Population,GDP PPP 2017,GDP PPP/capita 2017,Urban population %,Region,Income Group,IW,z_score,prev_ratio,next_ratio,year_gap,iqr_lower,iqr_upper,rolling_lower,rolling_upper,growth_rate,anomaly_methods
70,2004,Croatia,4304600,9.833295e+10,22843.69,54.147,Europe & Central Asia,High income,10136447.0,2.084098,NaN,1.336473,NaN,-3.954991e+05,9.736742e+06,5.313256e+06,1.495964e+07,NaN,2
103,2010,Denmark,5547683,2.819633e+11,50825.41,86.795,Europe & Central Asia,High income,14093663.0,2.744178,1.681724,1.563822,2.0,6.101846e+06,1.044722e+07,3.747410e+06,1.686617e+07,68.172383,3
116,2016,Estonia,1315790,4.211704e+10,32008.93,68.563,Europe & Central Asia,High income,32109695.0,2.129772,1.261403,1.262727,2.0,1.601645e+07,3.131726e+07,2.013870e+07,3.531863e+07,26.140267,2
180,2010,Ireland,4560155,2.436774e+11,53436.22,61.542,Europe & Central Asia,High income,23029496.0,2.086121,1.910499,1.461823,2.0,1.018037e+07,2.258163e+07,5.158224e+06,2.805005e+07,91.049924,3
203,1996,Japan,125757000,4.512009e+12,35878.79,78.145,East Asia & Pacific,High income,276340000.0,3.025864,1.125805,1.133191,1.0,2.021244e+08,2.665593e+08,2.166948e+08,2.924441e+08,12.580461,2
237,2020,Latvia,1900449,5.620686e+10,29575.57,68.315,Europe & Central Asia,High income,3244995.0,2.082578,2.388887,1.213461,2.0,4.102098e+05,2.829086e+06,-1.074731e+05,4.144817e+06,138.888713,3
313,2006,Portugal,10522288,3.317125e+11,31524.75,58.137,Europe & Central Asia,High income,37419554.0,2.170056,1.186385,2.761273,2.0,8.440674e+06,2.224598e+07,2.616639e+07,4.279398e+07,18.638525,2
338,2022,Serbia,6664449,1.391929e+11,20885.89,56.873,Europe & Central Asia,Upper middle income,175978055.0,2.241947,3.060968,NaN,2.0,3.622207e+07,6.857848e+07,-4.620422e+07,2.356324e+08,206.096835,3
358,2022,Slovenia,2111986,8.662358e+10,41015.23,55.751,Europe & Central Asia,High income,11800554.0,2.276709,1.477457,NaN,2.0,1.772097e+06,1.097043e+07,5.508582e+06,1.354656e+07,47.745701,2



===== 各国数据分布情况分析 =====

国家: Austria, 数据点数: 10
平均值: 24396081.40, 标准差: 6964804.78
最大Z-score: 1.70
最大年度增长率: 13.02%
最小年度增长率: -39.51%
潜在异常点:


,Year,IW
5,2012,16767513.0



国家: Belgium, 数据点数: 10
平均值: 60768433.40, 标准差: 11540179.67
最大Z-score: 1.93
最大年度增长率: 74.45%
最小年度增长率: -31.77%
潜在异常点:


...


国家: Slovenia, 数据点数: 10
平均值: 6665257.10, 标准差: 2255578.57
最大Z-score: 2.28
最大年度增长率: 47.75%
最小年度增长率: -25.68%
潜在异常点:


,Year,IW
358,2022,11800554.0



国家: Spain, 数据点数: 10
平均值: 95067837.20, 标准差: 7979316.05
最大Z-score: 1.97
最大年度增长率: 13.30%
最小年度增长率: -27.04%

国家: Sweden, 数据点数: 10
平均值: 127285687.10, 标准差: 28038989.24
最大Z-score: 1.34
最大年度增长率: 34.53%
最小年度增长率: -15.10%

国家: Turkiye, 数据点数: 9
平均值: 62367119.00, 标准差: 25641341.43
最大Z-score: 1.66
最大年度增长率: 70.18%
最小年度增长率: -27.83%
潜在异常点:


,Year,IW
383,2014,62283581.0



国家: United Kingdom, 数据点数: 8
平均值: 161932784.38, 标准差: 24449730.14
最大Z-score: 1.68
最大年度增长率: 11.90%
最小年度增长率: -27.73%

国家: United States, 数据点数: 29
平均值: 128936962.76, 标准差: 36864740.82
最大Z-score: 1.49
最大年度增长率: 17.38%
最小年度增长率: -24.27%


In [5]:
df_clean = detect_and_remove_anomalies(msw_df, target_col)
print(f"原始数据: {len(msw_df)} 行, 清洗后: {len(df_clean)} 行")

原始数据: 425 行, 清洗后: 416 行


In [6]:
train_df, predict_df = data_preprocessor.merge_features(df_clean)
print(f"用于训练的数据: {len(train_df)} 行, 预测的历史数据: {len(predict_df)} 行, 总共: {len(train_df)+len(predict_df)}行")

目标变量 'IW' 已进行 log1p 转换，生成列 'IW_log'
5431
用于训练的数据: 416 行, 预测的历史数据: 5015 行, 总共: 5431行


In [7]:
# 保存最终数据集
train_df.to_csv(
    Path(Config.PATH_CONFIG['features_dir']) / 'training_data.csv', 
    index=False,
    encoding='utf-8-sig'
)
predict_df.to_csv(
    Path(Config.PATH_CONFIG['features_dir']) / 'prediction_data.csv', 
    index=False,
    encoding='utf-8-sig'
)

In [8]:
# """加载2023-2050数据"""
# future_df = pd.read_excel(
#     Config.FEATURE_CONFIG['future_data_path'],
#     sheet_name=Config.FEATURE_CONFIG['future_sheet'],
#     usecols=Config.FEATURE_CONFIG['usecols']
# )

# """加载1990-2022数据"""
# historical_df = pd.read_excel(
#     Config.FEATURE_CONFIG['historical_data_path'],
#     sheet_name=Config.FEATURE_CONFIG['historical_sheet'],
#     usecols=Config.FEATURE_CONFIG['usecols']
# )
# future_prediction_df = data_preprocessor.process_future_data(historical_df, future_df)

# # 保存最终数据集
# future_prediction_df.to_csv(
#     Path(Config.PATH_CONFIG['features_dir']) / 'future_prediction_data.csv', 
#     index=False,
#     encoding='utf-8-sig'
# )